In [32]:
import pytesseract
from PIL import Image
from skimage.io import imread

# sample 1: classify different type of pan cards

image = imread('resources/dummy_pans/pan_old/oAbbot_Mcintyre.jpg')
to_search = ['S','i','g','n','a','t','u','r','e']

def classification_label(image, to_search):
    bboxes = pytesseract.image_to_boxes(image, output_type='dict')
    r = image.shape[0]
    c = image.shape[1]

    chars = bboxes['char']
    found_at = -1
    search_string_found = '0' #0 means not found, 1 means found
    for index, character in enumerate(chars):
        if character == to_search[0] and chars[index + 1] == to_search[1]:
            #run the search
            matched = 2 # already found 2
            for search_index in range(2, len(to_search)):
                if chars[index + search_index] == to_search[search_index]:
                    matched += 1
            if matched == len(to_search):
                found_at = index
                search_string_found = '1'
                break
                
    # divide the image into 16 parts and find in which part the signature is present.
    region_where_found = -1
    if found_at > -1:
        char_location_left = bboxes['left'][found_at]
        char_location_top = r-bboxes['top'][found_at]
        divider = 4    
        region_index = -1
        for ri in range(divider):
            for ci in range(divider):
                region_index+=1
                rmin = int((ri/divider)*r)
                rmax = int((ri/divider+1/divider)*r)
                cmin = int((ci/divider)*c)
                cmax = int((ci/divider+1/divider)*c)
                if char_location_top > rmin and char_location_top < rmax and char_location_left > cmin and char_location_left < cmax:
                    region_where_found = region_index
                    break  
                    
    return search_string_found + str(region_where_found)

In [33]:
classification_label(image, to_search)

'112'

In [35]:
from os import listdir
from os.path import isfile, join
import os  

mypath = '/Users/muthukrishnan/programming/ml_research/resources/dummy_pans/company'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

groups = {}
for file in onlyfiles[0:20]:
    try:
        label = classification_label(imread(join(mypath, file)), to_search)
        if label not in groups:
            groups[label] = []
        groups[label].append(file)
    except:
        print(file)
        
print(groups)

{'0-1': ['Pede_Blandit_Congue_Corporation.jpg', 'Vivamus_Non_Lorem_Consulting.jpg', 'Ullamcorper_Velit_In_Industries.jpg', 'Ligula_Associates.jpg', 'Vestibulum_Foundation.jpg', 'Rutrum_Magna_Cras_LLP.jpg', 'At_Egestas_A_Ltd.jpg', 'Nam_Ac_Ltd.jpg', 'Turpis_Egestas_Foundation.jpg', 'Nisi_Dictum_Augue_Inc..jpg', 'Lectus_Ante_Dictum_LLC.jpg', 'Scelerisque_Sed_Sapien_Limited.jpg', 'Sollicitudin_A_Company.jpg', 'Pellentesque_Pede_Blandit_Corporation.jpg', 'Magna_Malesuada_Foundation.jpg', 'Proin_Sed_Institute.jpg', 'Et_Arcu_Company.jpg', 'Tellus_Non_Magna_Inc..jpg', 'Lacinia_Mattis_Integer_Ltd.jpg', 'Aliquam_Ultrices_LLC.jpg']}
